Problem Statement: To build an application to classify the patients to be healthy or suffering from cardiovascular disease based on the given attributes.

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
data=pd.read_csv('cardio_train_modified_1000.csv',sep=';',header=0,index_col='id')

In [0]:
data.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
988,22469,1,155,69.0,130,80,2,2,0,0,1,0
989,14648,1,163,71.0,110,70,1,1,0,0,1,1
990,21901,1,165,70.0,120,80,1,1,0,0,1,0
991,14549,2,165,85.0,120,80,1,1,1,1,1,0
992,23393,1,155,62.0,120,80,1,1,0,0,1,0


In [0]:
data.shape

(1000, 12)

*Data set is Balanced or not?*

In [0]:
len(data[data.cardio==0])

521

In [0]:
len(data[data.cardio==1])

479

Inference: Data set is almost Balanced

*Data contains null values or not?*

In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 988 to 2394
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          1000 non-null   int64  
 1   gender       1000 non-null   int64  
 2   height       1000 non-null   int64  
 3   weight       1000 non-null   float64
 4   ap_hi        1000 non-null   int64  
 5   ap_lo        1000 non-null   int64  
 6   cholesterol  1000 non-null   int64  
 7   gluc         1000 non-null   int64  
 8   smoke        1000 non-null   int64  
 9   alco         1000 non-null   int64  
 10  active       1000 non-null   int64  
 11  cardio       1000 non-null   int64  
dtypes: float64(1), int64(11)
memory usage: 101.6 KB


Inference: data doesn't contain any null value

In [0]:
data.corr()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
age,1.000000,-0.035387,-0.099712,0.021426,0.126634,-0.000026,0.197988,0.127321,-0.043766,-0.025456,-0.000765,0.253741
gender,-0.035387,1.000000,0.529717,0.125945,0.061297,0.080293,-0.044352,-0.071817,0.316239,0.166434,0.031008,-0.022083
height,-0.099712,0.529717,1.000000,0.334461,0.047800,0.034423,-0.051899,-0.055928,0.176473,0.121742,0.004211,0.022921
weight,0.021426,0.125945,0.334461,1.000000,0.245063,-0.014340,0.159040,0.093750,0.076846,0.051993,-0.058702,0.194590
ap_hi,0.126634,0.061297,0.047800,0.245063,1.000000,0.221725,0.194754,0.072972,0.017422,0.034892,0.015569,0.398803
ap_lo,-0.000026,0.080293,0.034423,-0.014340,0.221725,1.000000,0.046451,0.016660,0.060656,0.011783,-0.029651,0.128263
cholesterol,0.197988,-0.044352,-0.051899,0.159040,0.194754,0.046451,1.000000,0.512579,0.007226,-0.019661,0.054798,0.225543
gluc,0.127321,-0.071817,-0.055928,0.093750,0.072972,0.016660,0.512579,1.000000,0.009238,-0.047638,0.021187,0.112975
smoke,-0.043766,0.316239,0.176473,0.076846,0.017422,0.060656,0.007226,0.009238,1.000000,0.409254,0.010895,-0.008920
alco,-0.025456,0.166434,0.121742,0.051993,0.034892,0.011783,-0.019661,-0.047638,0.409254,1.000000,0.055281,-0.031524


In [0]:
target=data['cardio']

In [0]:
feature=data.drop('cardio',axis=1)

*Check for the important features*

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
rfc=RandomForestClassifier()

In [0]:
featureImpModel=rfc.fit(feature,target)

In [0]:
featureImpModel.feature_importances_

array([0.23648529, 0.02353514, 0.15307556, 0.16839023, 0.19394808,
       0.10895812, 0.04392498, 0.02773235, 0.01054039, 0.00999154,
       0.02341833])

In [0]:
feature.columns

Index(['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol',
       'gluc', 'smoke', 'alco', 'active'],
      dtype='object')

In [0]:
feature_refined=feature.drop(['smoke', 'alco'],axis=1)

Inference: we can infer that columns 'smoke', 'alco' has little contribution to the model, hence can be dropped for now



In [0]:
feature_refined.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,active
id,,,,,,,,,
988,22469,1,155,69.0,130,80,2,2,1
989,14648,1,163,71.0,110,70,1,1,1
990,21901,1,165,70.0,120,80,1,1,1
991,14549,2,165,85.0,120,80,1,1,1
992,23393,1,155,62.0,120,80,1,1,1


Model building

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold,train_test_split,cross_val_score,GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier,RandomForestClassifier
from xgboost import XGBClassifier

In [0]:
X_train, X_test, y_train, y_test = train_test_split(feature_refined,target, test_size=0.25, random_state=25) 

In [0]:
pipe=Pipeline([('preprocessing',StandardScaler()),('classifier',SVC())])

In [0]:
search_space = [{'classifier': [SVC()],
                 'classifier__C': [1,10,100],
                 'classifier__kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
                {'classifier': [AdaBoostClassifier()],
                 'classifier__n_estimators': [10,50,100,150],
                 'classifier__learning_rate': [0.001,0.01,0.1,1,10]},
                {'classifier': [GradientBoostingClassifier()],
                 'classifier__n_estimators': [10,50,100,150],
                 'classifier__max_features': ['auto','log2'],
                 'classifier__learning_rate': [0.001,0.01,0.1,1,10]},
                {'classifier': [XGBClassifier()],
                 'classifier__max_depth': range(2,5,1),
                 'classifier__n_estimators': [10,50,100,150],
                 'classifier__learning_rate': [0.001,0.01,0.1,1,10]},
                {'classifier': [RandomForestClassifier()],
                 'classifier__max_depth': range(2,5,1),
                 'classifier__n_estimators': [10,50,100,150],
                 'classifier__max_features': ['auto','log2'],
                 'classifier__criterion': ['gini','entropy']}]

In [0]:
clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=10), verbose=3,n_jobs=-1)

In [0]:
best_model = clf.fit(X_train, y_train)

Fitting 10 folds for each of 180 candidates, totalling 1800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:   32.3s
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 1042 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 1800 out of 1800 | elapsed:  1.9min finished


In [0]:
best_model.best_estimator_.get_params()['classifier']

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=4, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=150,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
from sklearn.metrics import confusion_matrix,f1_score,roc_auc_score,accuracy_score

In [0]:
y_pred=best_model.predict(X_test)

In [0]:
confusion_matrix(y_test,y_pred)

array([[106,  17],
       [ 50,  77]])

In [0]:
roc_auc_score(y_test,y_pred)

0.7340439152423021

In [0]:
accuracy_score(y_test,y_pred)

0.732

In [0]:
best_model.predict(X_test)[113]

1

In [0]:
y_test.iloc[113]

1

Import total Data available and try to check the model *performance*

In [0]:
data=pd.read_csv('cardio_train.csv',sep=';',header=0,index_col='id')

In [0]:
target=data['cardio']

In [0]:
feature=data.drop('cardio',axis=1)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(feature,target, test_size=0.25, random_state=25) 

In [0]:
model=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=4, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=150,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=4, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=150,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
y_pred=model.predict(X_test)

In [0]:
confusion_matrix(y_test,y_pred)

array([[6865, 1863],
       [2942, 5656]])

In [0]:
accuracy_score(y_test,y_pred)

0.7226711300935011

In [0]:
roc_auc_score(y_test,y_pred)